# Reading Model Predictions and Bet365 Odds

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import warnings

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

csv_url = "https://docs.google.com/spreadsheets/d/1WfEG-1icUjj6k7TGePJQEXH-w0TLEIcN/export?format=csv"
uefa = pd.read_csv(csv_url, dtype={'FTHG': 'Int64', 'FTAG': 'Int64', 'HTHG': 'Int64', 'HTAG': 'Int64'})
uefa['FT'] = uefa['FTHG'].astype(str) + ' - ' + uefa['FTAG'].astype(str)
uefa['HT'] = '(' + uefa['HTHG'].astype(str) + '-' + uefa['HTAG'].astype(str) + ')'
uefa['FTTG'] = uefa['FTHG'] + uefa['FTAG']
uefa['HTTG'] = uefa['HTHG'] + uefa['HTAG']

predictions = pd.read_excel('C:/Users/99451/Desktop/MODEL/2025/dixon_coles_model_predictions/_predictions.xlsx')
bet365_odds = pd.read_excel('C:/Users/99451/Desktop/MODEL/2025/dixon_coles_model_predictions/final_odds.xlsx')
bet365_odds.tail()

,Home,Away,FT1,FTX,FT2,DC1X,DC12,DCX2,HT1,HTX,HT2,HT1X,HT12,HTX2,BTTS,OTTS,1.5O,1.5U,2.5O,2.5U,3.5O,3.5U,4.5O,4.5U,HT0.5O,HT0.5U,HT1.5O,HT1.5U
3525,AEK Athens FC,Athens Kallithea,1.14,8.50,13.00,1.04,1.08,5.50,1.53,2.88,13.0,1.01,1.40,2.38,2.5,1.50,1.14,5.50,1.50,2.50,2.25,1.57,3.75,1.25,1.25,3.75,2.20,1.62
3526,Brescia,Sampdoria,2.70,3.20,2.63,1.44,1.36,1.44,3.40,2.05,3.4,1.30,1.73,1.30,1.8,1.95,1.33,3.25,2.08,1.73,3.75,1.25,8.00,1.08,1.44,2.63,3.25,1.33
3527,PSG,St Etienne,1.10,9.50,23.00,1.01,1.07,7.00,1.36,3.60,13.0,1.01,1.29,3.00,2.0,1.75,1.07,9.00,1.25,4.00,1.67,2.20,2.38,1.57,1.14,5.50,1.80,2.05
3528,Napoli,Verona,1.22,5.75,15.00,1.03,1.14,4.00,1.67,2.63,10.0,1.03,1.44,2.10,2.2,1.62,1.20,4.50,1.67,2.20,2.63,1.50,4.50,1.20,1.30,3.40,2.38,1.53
3529,Alverca,Tondela,2.63,3.10,2.75,1.40,1.36,1.44,3.40,1.95,3.5,1.29,1.80,1.30,2.0,1.73,1.44,2.63,2.40,1.53,4.50,1.18,11.00,1.05,1.53,2.38,3.50,1.29


# Merging 2 DataFrames for similarity of values

In [2]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Function for fuzzy matching on both columns
def fuzzy_merge_on_two_columns(df1, df2, key1_home, key1_away, key2_home, key2_away, threshold=80):
    """
    Merge two DataFrames based on fuzzy matching of both Home and Away columns.
    - df1, df2: DataFrames to merge
    - key1_home, key1_away: column names for 'Home' and 'Away' in df1
    - key2_home, key2_away: column names for 'Home' and 'Away' in df2
    - threshold: minimum similarity score for a match
    """
    matches = []
    
    for i, row1 in df1.iterrows():
        home_team1, away_team1 = row1[key1_home], row1[key1_away]
        
        # Find the best match in df2 for both Home and Away teams
        best_match = None
        best_score = 0
        
        for j, row2 in df2.iterrows():
            home_team2, away_team2 = row2[key2_home], row2[key2_away]
            
            # Calculate similarity for both Home and Away columns
            home_score = fuzz.ratio(home_team1, home_team2)
            away_score = fuzz.ratio(away_team1, away_team2)
            
            # Average similarity score for the pair
            avg_score = (home_score + away_score) / 2
            
            # Check if this is the best match
            if avg_score >= threshold and avg_score > best_score:
                best_match = j
                best_score = avg_score
        
        # If a match was found above the threshold, save the indices
        if best_match is not None:
            matches.append((i, best_match))

    # Create matched DataFrames based on indices
    matched_df1 = df1.loc[[i for i, _ in matches]].reset_index(drop=True)
    matched_df2 = df2.loc[[j for _, j in matches]].reset_index(drop=True)
    
    # Concatenate the matched data side by side
    return pd.concat([matched_df1, matched_df2], axis=1, keys=["df1", "df2"])

# Use the function to merge
merged_df = fuzzy_merge_on_two_columns(predictions, bet365_odds, 'Home', 'Away', 'Home', 'Away', threshold=80)
merged_df.tail()

df1                                                                 \
      League         Home             Away    FT1    FTX    FT2  FTR   DC1X   
2938  Spain2        Elche    Real Zaragoza  52.67  27.44  19.89  1-0  80.11   
2939  Turkey    Sivasspor       Alanyaspor  33.05  36.70  30.25  1-1  69.75   
2940  Turkey  Trabzonspor      Antalyaspor  63.71  22.42  13.79  1-1  86.13   
2941  Turkey   Basaksehir      Galatasaray  22.94  22.73  54.22  1-1  45.67   
2942  Turkey    Gaziantep  Adana Demirspor  72.68  19.61   7.59  2-0  92.29   

                                                                            \
       DC12   DCX2   1.5O   2.5O   3.5U   4.5U   BTTS    HT1    HTX    HT2   
2938  72.56  47.33  66.28  39.22  80.83  92.13  42.45  46.81  43.25   9.80   
2939  63.30  66.95  67.36  36.81  82.60  93.12  47.37  13.84  49.45  36.65   
2940  77.50  36.21  88.62  68.06  53.39  72.30  63.06  35.97  46.02  17.92   
2941  77.16  76.95  93.04  76.99  42.13  61.66  74.92   9.80  26.48  60.75   
2942  80.27  27.20  85.70  63.41  58.62  76.74  51.61  58.60  32.85   7.63   

                                                                          \
      HTR HTDC1X HTDC12 HTDCX2 HT0.5O HT1.5U  H0.5O  A0.5O  H1.5O  A1.5O   
2938  0-0  90.06  56.61  53.05  63.70  72.42  76.86  54.70  43.00  18.83   
2939  0-0  63.29  50.49  86.10  59.83  74.15  67.06  65.19  30.48  28.45   
2940  0-0  81.99  53.89  63.94  66.12  66.80  90.87  66.96  69.14  30.41   
2941  0-1  36.28  70.55  87.23  84.73  32.49  79.98  91.27  47.86  70.13   
2942  1-0  91.45  66.23  40.48  76.59  52.42  91.63  54.08  71.03  18.36   

                            df2                                                \
      H2.5U  A2.5U         Home             Away   FT1  FTX   FT2  DC1X  DC12   
2938  81.79  95.37        Elche         Zaragoza  1.83  3.2  4.75  1.18  1.36   
2939  89.83  90.93    Sivasspor       Alanyaspor  2.75  3.4  2.35  1.57  1.30   
2940  56.88  89.78  Trabzonspor      Antalyaspor  1.53  4.2  5.00  1.17  1.22   
2941  77.93  55.66   Basaksehir      Galatasaray  4.75  4.1  1.57  2.30  1.22   
2942  54.54  95.44    Gaziantep  Adana Demirspor  1.44  4.5  5.50  1.13  1.17   

                                                                            \
      DCX2   HT1  HTX  HT2  HT1X  HT12  HTX2  BTTS  OTTS  1.5O  1.5U  2.5O   
2938  1.91  2.50  2.0  5.5  1.13  1.73  1.50  2.20  1.62  1.44  2.63  2.35   
2939  1.40  3.50  2.2  3.1  1.36  1.62  1.29  1.73  2.00  1.29  3.50  1.93   
2940  2.38  2.10  2.5  5.0  1.14  1.50  1.67  1.67  2.10  1.17  5.00  1.57   
2941  1.18  4.75  2.5  2.1  1.67  1.50  1.17  1.53  2.38  1.14  5.50  1.48   
2942  2.50  1.95  2.5  6.0  1.11  1.50  1.80  1.73  2.00  1.17  5.00  1.53   

                                                                 
      2.5U  3.5O  3.5U   4.5O  4.5U HT0.5O HT0.5U HT1.5O HT1.5U  
2938  1.57  4.33  1.20  10.00  1.06   1.53   2.38   3.50   1.29  
2939  1.93  3.25  1.33   6.50  1.11   1.40   2.75   2.75   1.40  
2940  2.35  2.38  1.53   4.33  1.20   1.29   3.50   2.38   1.53  
2941  2.60  2.10  1.67   3.75  1.25   1.25   3.75   2.20   1.62  
2942  2.40  2.38  1.53   4.00  1.22   1.29   3.50   2.25   1.57

# Scraping SoccerStats For Match Results

In [3]:
final =  pd.DataFrame()
liqa = ''
unique_leagues = predictions['League'].unique().tolist()

# Convert to lowercase and exclude UEFA competitions
uefa_list = ['unl', 'uel', 'ucl', 'ufcl']
unique_leagues = [league.lower() for league in unique_leagues if league.lower() not in uefa_list]

for i in unique_leagues:
    URL = "https://www.soccerstats.com/results.asp?league=" + i + "&pmtype=bydate"
    page = requests.get(URL)
    liqa = i
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="btable")
    sth = results.find_all("tr", class_="odd")
    sth


    date, league, home, away, ft, ht = [], [], [], [], [],[]
    for i in sth:
        date.append(i.find_all("td", align = 'right')[0].get_text(strip=True))
        league.append(liqa.capitalize())
        home.append(i.find_all("td", align = 'right')[1].get_text(strip=True))
        away.append(i.find("td", align = "left").get_text(strip = True))
        ft.append(i.find_all("td", align = 'center')[0].get_text(strip = True))
        try:
            ht.append(i.find_all("td", align = 'center')[2].get_text(strip = True))
        except IndexError as e:
            ht.append('NA')#print("Last output before error occurred:", i.find_all("td", align = 'center'))

    data = {'Date': date, 'League': league,'Home': home, 'Away': away, 'FT': ft, 'HT': ht}

# Create a DataFrame from the dictionary
    df = pd.DataFrame(data)

# Replace empty strings with NaN
    #next_df = df[(df['Date'] == formatted_date) & (df['HT'] == '')]
    df.replace('', pd.NA, inplace=True)

# Drop rows with NaN values
    df_cleaned = df.dropna()

#For Half-Time Results
    hthg, htag = [], []
    for i in df_cleaned['HT']:
        if i == 'NA':
            hthg.append('NA')
            htag.append('NA')
        elif i == '+' or i == '-':
            hthg.append('NA')
            htag.append('NA')
        else:
            try:
                hthg.append(int(i[1]))
                htag.append(int(i[3]))
            except IndexError as e:
                print("Last output before error occurred:", i)



#For Full-Time Results
    hg, ag, tg = [], [], []
    for i in df_cleaned['FT']:
        if len(i) < 5 or ':' in i:
            hg.append('NA')
            ag.append('NA')
            tg.append('NA')
        else:
            try:
                hghg = int(i.split(' - ')[0])
                hg.append(hghg)
                agag = int(i.split(' - ')[1])
                ag.append(agag)
                tg.append(hghg + agag)
            except:
                print(hghg + agag)

    
    df_cleaned['FTHG'], df_cleaned['FTAG'], df_cleaned['FTTG'] = hg, ag, tg
    df_cleaned['HTHG'], df_cleaned['HTAG'] = hthg, htag
    df_cleaned['HTTG'] = df_cleaned['HTHG'] + df_cleaned['HTAG']
    
    final = pd.concat([final, df_cleaned], ignore_index=True)
    
final = final[final['HT'] != 'NA']

# Example: Fix the 'date' column by removing the weekday and extra punctuation
final['Date'] = final['Date'].str.extract(r'(\d{1,2} \w{3})')  # Extract day and month part
final['Date'] = final['Date'] + ' 2024'  # Append the year

# Convert to datetime format
final['Date'] = pd.to_datetime(final['Date'], format='%d %b %Y', errors='coerce')

# Filter rows before September 6th, 2024
final_filtered = final[final['Date'] >= pd.Timestamp('2024-09-17')]

# Align columns of uefa to match final_filtered
uefa = uefa[final_filtered.columns]

# Concatenate
final_filtered = pd.concat([uefa, final_filtered], ignore_index=True)

combined = pd.concat([final_filtered.head(), final_filtered.tail()])

combined

,Date,League,Home,Away,FT,HT,FTHG,FTAG,FTTG,HTHG,HTAG,HTTG
0,9/17/2024,UCL,Juventus,PSV,3 - 1,(2-0),3.0,1.0,4.0,2.0,0.0,2.0
1,9/17/2024,UCL,Young Boys,Aston Villa,0 - 3,(0-2),0.0,3.0,3.0,0.0,2.0,2.0
2,9/17/2024,UCL,Bayern,Dinamo Zagreb,9 - 2,(3-0),9.0,2.0,11.0,3.0,0.0,3.0
3,9/17/2024,UCL,Milan,Liverpool,1 - 3,(1-2),1.0,3.0,4.0,1.0,2.0,3.0
4,9/17/2024,UCL,Real Madrid,Stuttgart,3 - 1,(0-0),3.0,1.0,4.0,0.0,0.0,0.0
7547,2024-12-21 00:00:00,Scotland4,Stirling Albion,Stranraer,1 - 1,(0-0),1.0,1.0,2.0,0.0,0.0,0.0
7548,2024-12-28 00:00:00,Scotland4,Elgin City,Peterhead,2 - 0,(1-0),2.0,0.0,2.0,1.0,0.0,1.0
7549,2024-12-28 00:00:00,Scotland4,Forfar Athletic,Stirling Albion,2 - 2,(1-0),2.0,2.0,4.0,1.0,0.0,1.0
7550,2024-12-28 00:00:00,Scotland4,Spartans,Edinburgh City,1 - 0,(0-0),1.0,0.0,1.0,0.0,0.0,0.0
7551,2024-12-28 00:00:00,Scotland4,Stranraer,Clyde,1 - 1,(0-1),1.0,1.0,2.0,0.0,1.0,1.0


# Merging with Predictions + Odds Dataframes

In [4]:
# Flatten multi-level columns
merged_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in merged_df.columns]

# Rename two columns
merged_df = merged_df.rename(columns={'df1_Home': 'Home', 'df1_Away': 'Away'})

# Merge on multiple columns
final_df = pd.merge(merged_df, final_filtered, on=['Home', 'Away'], how='inner')

# Drop duplicates based on the 'Home' and 'Away' columns
final_df_unique = final_df.drop_duplicates(subset=['Home', 'Away'])
final_df_unique = final_df_unique.dropna()

# Reset the index afterward
final_df_unique.reset_index(drop=True, inplace=True)

print('Number of games matched: ', len(final_df_unique))
final_df_unique.tail()

Number of games matched:  2626


,df1_League,Home,Away,df1_FT1,df1_FTX,df1_FT2,df1_FTR,df1_DC1X,df1_DC12,df1_DCX2,df1_1.5O,df1_2.5O,df1_3.5U,df1_4.5U,df1_BTTS,df1_HT1,df1_HTX,df1_HT2,df1_HTR,df1_HTDC1X,df1_HTDC12,df1_HTDCX2,df1_HT0.5O,df1_HT1.5U,df1_H0.5O,df1_A0.5O,df1_H1.5O,df1_A1.5O,df1_H2.5U,df1_A2.5U,df2_Home,df2_Away,df2_FT1,df2_FTX,df2_FT2,df2_DC1X,df2_DC12,df2_DCX2,df2_HT1,df2_HTX,df2_HT2,df2_HT1X,df2_HT12,df2_HTX2,df2_BTTS,df2_OTTS,df2_1.5O,df2_1.5U,df2_2.5O,df2_2.5U,df2_3.5O,df2_3.5U,df2_4.5O,df2_4.5U,df2_HT0.5O,df2_HT0.5U,df2_HT1.5O,df2_HT1.5U,Date,League,FT,HT,FTHG,FTAG,FTTG,HTHG,HTAG,HTTG
2621,Switzerland,Lugano,Lausanne Sport,48.76,24.26,26.98,1-1,73.02,75.74,51.24,76.17,52.30,69.89,85.25,54.53,46.63,32.38,20.63,1-0,79.01,67.26,53.01,78.85,57.00,80.31,68.11,48.32,31.67,77.67,89.15,Lugano,Lausanne,2.20,3.5,3.00,1.36,1.29,1.67,2.88,2.25,3.50,1.29,1.57,1.40,1.62,2.20,1.22,4.00,1.73,2.08,2.75,1.40,5.50,1.14,1.33,3.25,2.63,1.44,2024-12-15 00:00:00,Switzerland,1 - 4,(0-2),1.0,4.0,5.0,0.0,2.0,2.0
2622,Turkey,Hatayspor,Goztepe,17.23,27.10,55.65,1-1,44.33,72.88,82.75,80.37,54.61,67.71,83.71,55.63,30.98,43.25,25.53,0-0,74.23,56.51,68.78,74.75,50.87,62.89,84.65,26.11,55.90,92.11,71.04,Hatayspor,Goztepe,2.88,3.5,2.35,1.57,1.29,1.40,3.50,2.20,3.00,1.36,1.62,1.29,1.73,2.00,1.25,3.75,1.85,2.00,3.00,1.36,6.00,1.13,1.40,2.75,2.75,1.40,2024-12-15 00:00:00,Turkey,1 - 1,(0-1),1.0,1.0,2.0,0.0,1.0,1.0
2623,Turkey,Antalyaspor,Kayserispor,51.51,26.04,22.43,1-1,77.55,73.94,48.47,86.08,63.68,58.42,76.57,64.76,28.67,46.54,24.68,0-0,75.21,53.35,71.22,69.31,59.55,86.34,72.47,59.16,36.97,67.88,85.91,Antalyaspor,Kayserispor,2.10,3.5,3.30,1.33,1.29,1.70,2.75,2.20,3.75,1.25,1.62,1.40,1.67,2.10,1.25,3.75,1.85,2.00,3.00,1.36,6.00,1.13,1.36,3.00,2.75,1.40,2024-12-15 00:00:00,Turkey,2 - 0,(1-0),2.0,0.0,2.0,1.0,0.0,1.0
2624,Turkey,Fenerbahce,Basaksehir,73.45,17.43,8.90,2-0,90.88,82.35,26.33,89.73,71.47,49.12,68.45,58.88,63.97,29.71,4.71,1-0,93.68,68.68,34.42,78.42,46.85,93.45,61.32,76.00,24.65,47.88,92.60,Fenerbahce,Basaksehir,1.40,5.5,6.00,1.13,1.14,2.75,1.91,2.50,6.50,1.10,1.50,1.80,1.83,1.83,1.18,4.50,1.57,2.35,2.50,1.50,4.33,1.20,1.30,3.40,2.38,1.53,2024-12-15 00:00:00,Turkey,3 - 1,(1-0),3.0,1.0,4.0,1.0,0.0,1.0
2625,Scotland2,Hamilton,Partick Thistle,0.24,0.47,40.89,1-8,0.71,41.13,41.36,41.56,41.44,0.61,1.83,31.40,0.05,0.18,7.15,0-4,0.23,7.20,7.33,7.37,0.11,31.41,41.58,17.08,41.50,34.59,0.49,Hamilton,Partick Thistle,2.90,3.6,2.10,1.60,1.22,1.30,3.50,2.20,2.75,1.40,1.62,1.29,1.67,2.10,1.22,4.00,1.75,2.05,2.75,1.40,5.00,1.14,1.36,3.00,2.63,1.44,2024-09-28 00:00:00,Scotland2,1 - 0,(0-0),1.0,0.0,1.0,0.0,0.0,0.0


# Creating Results Columns

In [5]:
import numpy as np

# Add the new columns based on the condition
final_df_unique['FT1'] = np.where(final_df_unique['FTHG'] > final_df_unique['FTAG'], 1, 0)
final_df_unique['FTX'] = np.where(final_df_unique['FTHG'] == final_df_unique['FTAG'], 1, 0)
final_df_unique['FT2'] = np.where(final_df_unique['FTHG'] < final_df_unique['FTAG'], 1, 0)

final_df_unique['FT1X'] = np.where(final_df_unique['FTHG'] >= final_df_unique['FTAG'], 1, 0)
final_df_unique['FT12'] = np.where(final_df_unique['FTX'] == 0, 1, 0)
final_df_unique['FTX2'] = np.where(final_df_unique['FTHG'] <= final_df_unique['FTAG'], 1, 0)

final_df_unique['1.5O'] = np.where(final_df_unique['FTTG'] > 1.5, 1, 0)
final_df_unique['1.5U'] = np.where(final_df_unique['FTTG'] < 1.5, 1, 0)
final_df_unique['2.5O'] = np.where(final_df_unique['FTTG'] > 2.5, 1, 0)
final_df_unique['2.5U'] = np.where(final_df_unique['FTTG'] < 2.5, 1, 0)
final_df_unique['3.5O'] = np.where(final_df_unique['FTTG'] > 3.5, 1, 0)
final_df_unique['3.5U'] = np.where(final_df_unique['FTTG'] < 3.5, 1, 0)
final_df_unique['4.5O'] = np.where(final_df_unique['FTTG'] > 4.5, 1, 0)
final_df_unique['4.5U'] = np.where(final_df_unique['FTTG'] < 4.5, 1, 0)

final_df_unique['BTTS'] = np.where((final_df_unique['FTHG'] != 0) & (final_df_unique['FTAG'] != 0), 1, 0)
final_df_unique['OTTS'] = np.where(final_df_unique['BTTS'] == 0, 1, 0)

final_df_unique['HT1'] = np.where(final_df_unique['HTHG'] > final_df_unique['HTAG'], 1, 0)
final_df_unique['HTX'] = np.where(final_df_unique['HTHG'] == final_df_unique['HTAG'], 1, 0)
final_df_unique['HT2'] = np.where(final_df_unique['HTHG'] < final_df_unique['HTAG'], 1, 0)

final_df_unique['HT1X'] = np.where(final_df_unique['HTHG'] >= final_df_unique['HTAG'], 1, 0)
final_df_unique['HT12'] = np.where(final_df_unique['HTX'] == 0, 1, 0)
final_df_unique['HTX2'] = np.where(final_df_unique['HTHG'] <= final_df_unique['HTAG'], 1, 0)

final_df_unique['HT0.5O'] = np.where(final_df_unique['HTTG'] > 0.5, 1, 0)
final_df_unique['HT0.5U'] = np.where(final_df_unique['HTTG'] < 0.5, 1, 0)
final_df_unique['HT1.5O'] = np.where(final_df_unique['HTTG'] > 1.5, 1, 0)
final_df_unique['HT1.5U'] = np.where(final_df_unique['HTTG'] < 1.5, 1, 0)

print('Games Found: ', len(final_df_unique))
final_df_unique.tail()

Games Found:  2626


,df1_League,Home,Away,df1_FT1,df1_FTX,df1_FT2,df1_FTR,df1_DC1X,df1_DC12,df1_DCX2,df1_1.5O,df1_2.5O,df1_3.5U,df1_4.5U,df1_BTTS,df1_HT1,df1_HTX,df1_HT2,df1_HTR,df1_HTDC1X,df1_HTDC12,df1_HTDCX2,df1_HT0.5O,df1_HT1.5U,df1_H0.5O,df1_A0.5O,df1_H1.5O,df1_A1.5O,df1_H2.5U,df1_A2.5U,df2_Home,df2_Away,df2_FT1,df2_FTX,df2_FT2,df2_DC1X,df2_DC12,df2_DCX2,df2_HT1,df2_HTX,df2_HT2,df2_HT1X,df2_HT12,df2_HTX2,df2_BTTS,df2_OTTS,df2_1.5O,df2_1.5U,df2_2.5O,df2_2.5U,df2_3.5O,df2_3.5U,df2_4.5O,df2_4.5U,df2_HT0.5O,df2_HT0.5U,df2_HT1.5O,df2_HT1.5U,Date,League,FT,HT,FTHG,FTAG,FTTG,HTHG,HTAG,HTTG,FT1,FTX,FT2,FT1X,FT12,FTX2,1.5O,1.5U,2.5O,2.5U,3.5O,3.5U,4.5O,4.5U,BTTS,OTTS,HT1,HTX,HT2,HT1X,HT12,HTX2,HT0.5O,HT0.5U,HT1.5O,HT1.5U
2621,Switzerland,Lugano,Lausanne Sport,48.76,24.26,26.98,1-1,73.02,75.74,51.24,76.17,52.30,69.89,85.25,54.53,46.63,32.38,20.63,1-0,79.01,67.26,53.01,78.85,57.00,80.31,68.11,48.32,31.67,77.67,89.15,Lugano,Lausanne,2.20,3.5,3.00,1.36,1.29,1.67,2.88,2.25,3.50,1.29,1.57,1.40,1.62,2.20,1.22,4.00,1.73,2.08,2.75,1.40,5.50,1.14,1.33,3.25,2.63,1.44,2024-12-15 00:00:00,Switzerland,1 - 4,(0-2),1.0,4.0,5.0,0.0,2.0,2.0,0,0,1,0,1,1,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,1,1,0,1,0
2622,Turkey,Hatayspor,Goztepe,17.23,27.10,55.65,1-1,44.33,72.88,82.75,80.37,54.61,67.71,83.71,55.63,30.98,43.25,25.53,0-0,74.23,56.51,68.78,74.75,50.87,62.89,84.65,26.11,55.90,92.11,71.04,Hatayspor,Goztepe,2.88,3.5,2.35,1.57,1.29,1.40,3.50,2.20,3.00,1.36,1.62,1.29,1.73,2.00,1.25,3.75,1.85,2.00,3.00,1.36,6.00,1.13,1.40,2.75,2.75,1.40,2024-12-15 00:00:00,Turkey,1 - 1,(0-1),1.0,1.0,2.0,0.0,1.0,1.0,0,1,0,1,0,1,1,0,0,1,0,1,0,1,1,0,0,0,1,0,1,1,1,0,0,1
2623,Turkey,Antalyaspor,Kayserispor,51.51,26.04,22.43,1-1,77.55,73.94,48.47,86.08,63.68,58.42,76.57,64.76,28.67,46.54,24.68,0-0,75.21,53.35,71.22,69.31,59.55,86.34,72.47,59.16,36.97,67.88,85.91,Antalyaspor,Kayserispor,2.10,3.5,3.30,1.33,1.29,1.70,2.75,2.20,3.75,1.25,1.62,1.40,1.67,2.10,1.25,3.75,1.85,2.00,3.00,1.36,6.00,1.13,1.36,3.00,2.75,1.40,2024-12-15 00:00:00,Turkey,2 - 0,(1-0),2.0,0.0,2.0,1.0,0.0,1.0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,0,1,1,0,0,1,1,0,1,0,0,1
2624,Turkey,Fenerbahce,Basaksehir,73.45,17.43,8.90,2-0,90.88,82.35,26.33,89.73,71.47,49.12,68.45,58.88,63.97,29.71,4.71,1-0,93.68,68.68,34.42,78.42,46.85,93.45,61.32,76.00,24.65,47.88,92.60,Fenerbahce,Basaksehir,1.40,5.5,6.00,1.13,1.14,2.75,1.91,2.50,6.50,1.10,1.50,1.80,1.83,1.83,1.18,4.50,1.57,2.35,2.50,1.50,4.33,1.20,1.30,3.40,2.38,1.53,2024-12-15 00:00:00,Turkey,3 - 1,(1-0),3.0,1.0,4.0,1.0,0.0,1.0,1,0,0,1,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,1,0,1,0,0,1
2625,Scotland2,Hamilton,Partick Thistle,0.24,0.47,40.89,1-8,0.71,41.13,41.36,41.56,41.44,0.61,1.83,31.40,0.05,0.18,7.15,0-4,0.23,7.20,7.33,7.37,0.11,31.41,41.58,17.08,41.50,34.59,0.49,Hamilton,Partick Thistle,2.90,3.6,2.10,1.60,1.22,1.30,3.50,2.20,2.75,1.40,1.62,1.29,1.67,2.10,1.22,4.00,1.75,2.05,2.75,1.40,5.00,1.14,1.36,3.00,2.63,1.44,2024-09-28 00:00:00,Scotland2,1 - 0,(0-0),1.0,0.0,1.0,0.0,0.0,0.0,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1


# Creating Profit Columns for Initial Model Predictions

In [6]:
ft1p, ftxp, ft2p, ft1xp, ft12p, ftx2p = [], [], [], [], [], []
over15, under15, over25, under25, over35, under35, over45, under45 = [], [], [], [], [], [], [], []
btts, otts, ht1p, htxp, ht2p, ht1xp, ht12p, htx2p = [], [], [], [], [], [], [], []
htover05, htunder05, htover15, htunder15 = [], [], [], []

#if prediction == result -> coefficient, elif prediction != result -> 0, else -> -1
for i in range(len(final_df_unique)):
    ft_list = [final_df_unique['df1_FT1'].iloc[i], final_df_unique['df1_FTX'].iloc[i], final_df_unique['df1_FT2'].iloc[i]]
    if (ft_list.index(max(ft_list)) == 0) and (final_df_unique['FT1'].iloc[i] == 1):
        ft1p.append(final_df_unique['df2_FT1'].iloc[i])
    elif (ft_list.index(max(ft_list)) == 0) and (final_df_unique['FT1'].iloc[i] != 1):
        ft1p.append(0)
    else:
        ft1p.append(-1)
    
    if (ft_list.index(max(ft_list)) == 1) and (final_df_unique['FTX'].iloc[i] == 1):
        ftxp.append(final_df_unique['df2_FTX'].iloc[i])
    elif (ft_list.index(max(ft_list)) == 1) and (final_df_unique['FTX'].iloc[i] != 1):
        ftxp.append(0)
    else:
        ftxp.append(-1)
    
    if (ft_list.index(max(ft_list)) == 2) and (final_df_unique['FT2'].iloc[i] == 1):
        ft2p.append(final_df_unique['df2_FT2'].iloc[i])
    elif (ft_list.index(max(ft_list)) == 2) and (final_df_unique['FT2'].iloc[i] != 1):
        ft2p.append(0)
    else:
        ft2p.append(-1)

    dc_list = [final_df_unique['df1_DC1X'].iloc[i], final_df_unique['df1_DC12'].iloc[i], final_df_unique['df1_DCX2'].iloc[i]]
    if (dc_list.index(max(dc_list)) == 0) and (final_df_unique['FT1X'].iloc[i] == 1):
        ft1xp.append(final_df_unique['df2_DC1X'].iloc[i])
    elif (dc_list.index(max(dc_list)) == 0) and (final_df_unique['FT1X'].iloc[i] != 1):
        ft1xp.append(0)
    else:
        ft1xp.append(-1)
    
    if (dc_list.index(max(dc_list)) == 1) and (final_df_unique['FT12'].iloc[i] == 1):
        ft12p.append(final_df_unique['df2_DC12'].iloc[i])
    elif (dc_list.index(max(dc_list)) == 1) and (final_df_unique['FT12'].iloc[i] != 1):
        ft12p.append(0)
    else:
        ft12p.append(-1)
    
    if (dc_list.index(max(dc_list)) == 2) and (final_df_unique['FTX2'].iloc[i] == 1):
        ftx2p.append(final_df_unique['df2_DCX2'].iloc[i])
    elif (dc_list.index(max(dc_list)) == 2) and (final_df_unique['FTX2'].iloc[i] != 1):
        ftx2p.append(0)
    else:
        ftx2p.append(-1)
    
    if (final_df_unique['df1_1.5O'].iloc[i] >= 50) and (final_df_unique['1.5O'].iloc[i] == 1):
        over15.append(final_df_unique['df2_1.5O'].iloc[i])
    elif (final_df_unique['df1_1.5O'].iloc[i] >= 50) and (final_df_unique['1.5O'].iloc[i] != 1):
        over15.append(0)
    else:
        over15.append(-1)
    
    if (final_df_unique['df1_1.5O'].iloc[i] < 50) and (final_df_unique['1.5O'].iloc[i] != 1):
        under15.append(final_df_unique['df2_1.5U'].iloc[i])
    elif (final_df_unique['df1_1.5O'].iloc[i] < 50) and (final_df_unique['1.5O'].iloc[i] == 1):
        under15.append(0)
    else:
        under15.append(-1)
    
    if (final_df_unique['df1_2.5O'].iloc[i] >= 50) and (final_df_unique['2.5O'].iloc[i] == 1):
        over25.append(final_df_unique['df2_2.5O'].iloc[i])
    elif (final_df_unique['df1_2.5O'].iloc[i] >= 50) and (final_df_unique['2.5O'].iloc[i] != 1):
        over25.append(0)
    else:
        over25.append(-1)
    
    if (final_df_unique['df1_2.5O'].iloc[i] < 50) and (final_df_unique['2.5O'].iloc[i] != 1):
        under25.append(final_df_unique['df2_2.5U'].iloc[i])
    elif (final_df_unique['df1_2.5O'].iloc[i] < 50) and (final_df_unique['2.5O'].iloc[i] == 1):
        under25.append(0)
    else:
        under25.append(-1)
    
    if (final_df_unique['df1_3.5U'].iloc[i] <= 50) and (final_df_unique['3.5O'].iloc[i] == 1):
        over35.append(final_df_unique['df2_3.5O'].iloc[i])
    elif (final_df_unique['df1_3.5U'].iloc[i] <= 50) and (final_df_unique['3.5O'].iloc[i] != 1):
        over35.append(0)
    else:
        over35.append(-1)
    
    if (final_df_unique['df1_3.5U'].iloc[i] > 50) and (final_df_unique['3.5O'].iloc[i] != 1):
        under35.append(final_df_unique['df2_3.5U'].iloc[i])
    elif (final_df_unique['df1_3.5U'].iloc[i] > 50) and (final_df_unique['3.5O'].iloc[i] == 1):
        under35.append(0)
    else:
        under35.append(-1)
    
    if (final_df_unique['df1_4.5U'].iloc[i] <= 50) and (final_df_unique['4.5O'].iloc[i] == 1):
        over45.append(final_df_unique['df2_4.5O'].iloc[i])
    elif (final_df_unique['df1_4.5U'].iloc[i] <= 50) and (final_df_unique['4.5O'].iloc[i] != 1):
        over45.append(0)
    else:
        over45.append(-1)
    
    if (final_df_unique['df1_4.5U'].iloc[i] > 50) and (final_df_unique['4.5O'].iloc[i] != 1):
        under45.append(final_df_unique['df2_4.5U'].iloc[i])
    elif (final_df_unique['df1_4.5U'].iloc[i] > 50) and (final_df_unique['4.5O'].iloc[i] == 1):
        under45.append(0)
    else:
        under45.append(-1)
    
    if (final_df_unique['df1_BTTS'].iloc[i] >= 50) and (final_df_unique['BTTS'].iloc[i] == 1):
        btts.append(final_df_unique['df2_BTTS'].iloc[i])
    elif (final_df_unique['df1_BTTS'].iloc[i] >= 50) and (final_df_unique['BTTS'].iloc[i] != 1):
        btts.append(0)
    else:
        btts.append(-1)
    
    if (final_df_unique['df1_BTTS'].iloc[i] < 50) and (final_df_unique['BTTS'].iloc[i] != 1):
        otts.append(final_df_unique['df2_OTTS'].iloc[i])
    elif (final_df_unique['df1_BTTS'].iloc[i] < 50) and (final_df_unique['BTTS'].iloc[i] == 1):
        otts.append(0)
    else:
        otts.append(-1)
    
    ht_list = [final_df_unique['df1_HT1'].iloc[i], final_df_unique['df1_HTX'].iloc[i], final_df_unique['df1_HT2'].iloc[i]]
    if (ht_list.index(max(ht_list)) == 0) and (final_df_unique['HT1'].iloc[i] == 1):
        ht1p.append(final_df_unique['df2_HT1'].iloc[i])
    elif (ht_list.index(max(ht_list)) == 0) and (final_df_unique['HT1'].iloc[i] != 1):
        ht1p.append(0)
    else:
        ht1p.append(-1)
    
    if (ht_list.index(max(ht_list)) == 1) and (final_df_unique['HTX'].iloc[i] == 1):
        htxp.append(final_df_unique['df2_HTX'].iloc[i])
    elif (ht_list.index(max(ht_list)) == 1) and (final_df_unique['HTX'].iloc[i] != 1):
        htxp.append(0)
    else:
        htxp.append(-1)
    
    if (ht_list.index(max(ht_list)) == 2) and (final_df_unique['HT2'].iloc[i] == 1):
        ht2p.append(final_df_unique['df2_HT2'].iloc[i])
    elif (ht_list.index(max(ht_list)) == 2) and (final_df_unique['HT2'].iloc[i] != 1):
        ht2p.append(0)
    else:
        ht2p.append(-1)
    
    htdc_list = [final_df_unique['df1_HTDC1X'].iloc[i], final_df_unique['df1_HTDC12'].iloc[i], final_df_unique['df1_HTDCX2'].iloc[i]]
    if (htdc_list.index(max(htdc_list)) == 0) and (final_df_unique['HT1X'].iloc[i] == 1):
        ht1xp.append(final_df_unique['df2_HT1X'].iloc[i])
    elif (htdc_list.index(max(htdc_list)) == 0) and (final_df_unique['HT1X'].iloc[i] != 1):
        ht1xp.append(0)
    else:
        ht1xp.append(-1)
    
    if (htdc_list.index(max(htdc_list)) == 1) and (final_df_unique['HT12'].iloc[i] == 1):
        ht12p.append(final_df_unique['df2_HT12'].iloc[i])
    elif (htdc_list.index(max(htdc_list)) == 1) and (final_df_unique['HT12'].iloc[i] != 1):
        ht12p.append(0)
    else:
        ht12p.append(-1)
    
    if (htdc_list.index(max(htdc_list)) == 2) and (final_df_unique['HTX2'].iloc[i] == 1):
        htx2p.append(final_df_unique['df2_HTX2'].iloc[i])
    elif (htdc_list.index(max(htdc_list)) == 2) and (final_df_unique['HTX2'].iloc[i] != 1):
        htx2p.append(0)
    else:
        htx2p.append(-1)
    
    if (final_df_unique['df1_HT0.5O'].iloc[i] >= 50) and (final_df_unique['HT0.5O'].iloc[i] == 1):
        htover05.append(final_df_unique['df2_HT0.5O'].iloc[i])
    elif (final_df_unique['df1_HT0.5O'].iloc[i] >= 50) and (final_df_unique['HT0.5O'].iloc[i] != 1):
        htover05.append(0)
    else:
        htover05.append(-1)
    
    if (final_df_unique['df1_HT0.5O'].iloc[i] < 50) and (final_df_unique['HT0.5O'].iloc[i] != 1):
        htunder05.append(final_df_unique['df2_HT0.5U'].iloc[i])
    elif (final_df_unique['df1_HT0.5O'].iloc[i] < 50) and (final_df_unique['HT0.5O'].iloc[i] == 1):
        htunder05.append(0)
    else:
        htunder05.append(-1)
    
    if (final_df_unique['df1_HT1.5U'].iloc[i] < 50) and (final_df_unique['HT1.5O'].iloc[i] == 1):
        htover15.append(final_df_unique['df2_HT1.5O'].iloc[i])
    elif (final_df_unique['df1_HT1.5U'].iloc[i] < 50) and (final_df_unique['HT1.5O'].iloc[i] != 1):
        htover15.append(0)
    else:
        htover15.append(-1)
    
    if (final_df_unique['df1_HT1.5U'].iloc[i] >= 50) and (final_df_unique['HT1.5O'].iloc[i] != 1):
        htunder15.append(final_df_unique['df2_HT1.5U'].iloc[i])
    elif (final_df_unique['df1_HT1.5U'].iloc[i] >= 50) and (final_df_unique['HT1.5O'].iloc[i] == 1):
        htunder15.append(0)
    else:
        htunder15.append(-1)

final_df_unique['FT1P'], final_df_unique['FTXP'], final_df_unique['FT2P'] = ft1p, ftxp, ft2p
final_df_unique['FT1XP'], final_df_unique['FT12P'], final_df_unique['FTX2P'] = ft1xp, ft12p, ftx2p
final_df_unique['1.5OP'], final_df_unique['1.5UP'], final_df_unique['2.5OP'], final_df_unique['2.5UP'] = over15, under15, over25, under25
final_df_unique['3.5OP'], final_df_unique['3.5UP'], final_df_unique['4.5OP'], final_df_unique['4.5UP'] = over35, under35, over45, under45
final_df_unique['BTTSP'], final_df_unique['OTTSP'] = btts, otts
final_df_unique['HT1P'], final_df_unique['HTXP'], final_df_unique['HT2P'] = ht1p, htxp, ht2p
final_df_unique['HT1XP'], final_df_unique['HT12P'], final_df_unique['HTX2P'] = ht1xp, ht12p, htx2p
final_df_unique['HT0.5OP'], final_df_unique['HT0.5UP'] = htover05, htunder05
final_df_unique['HT1.5OP'], final_df_unique['HT1.5UP'] = htover15, htunder15

print('Games Found: ', len(final_df_unique))
final_df_unique.tail()

Games Found:  2626


,df1_League,Home,Away,df1_FT1,df1_FTX,df1_FT2,df1_FTR,df1_DC1X,df1_DC12,df1_DCX2,df1_1.5O,df1_2.5O,df1_3.5U,df1_4.5U,df1_BTTS,df1_HT1,df1_HTX,df1_HT2,df1_HTR,df1_HTDC1X,df1_HTDC12,df1_HTDCX2,df1_HT0.5O,df1_HT1.5U,df1_H0.5O,df1_A0.5O,df1_H1.5O,df1_A1.5O,df1_H2.5U,df1_A2.5U,df2_Home,df2_Away,df2_FT1,df2_FTX,df2_FT2,df2_DC1X,df2_DC12,df2_DCX2,df2_HT1,df2_HTX,df2_HT2,df2_HT1X,df2_HT12,df2_HTX2,df2_BTTS,df2_OTTS,df2_1.5O,df2_1.5U,df2_2.5O,df2_2.5U,df2_3.5O,df2_3.5U,df2_4.5O,df2_4.5U,df2_HT0.5O,df2_HT0.5U,df2_HT1.5O,df2_HT1.5U,Date,League,FT,HT,FTHG,FTAG,FTTG,HTHG,HTAG,HTTG,FT1,FTX,FT2,FT1X,FT12,FTX2,1.5O,1.5U,2.5O,2.5U,3.5O,3.5U,4.5O,4.5U,BTTS,OTTS,HT1,HTX,HT2,HT1X,HT12,HTX2,HT0.5O,HT0.5U,HT1.5O,HT1.5U,FT1P,FTXP,FT2P,FT1XP,FT12P,FTX2P,1.5OP,1.5UP,2.5OP,2.5UP,3.5OP,3.5UP,4.5OP,4.5UP,BTTSP,OTTSP,HT1P,HTXP,HT2P,HT1XP,HT12P,HTX2P,HT0.5OP,HT0.5UP,HT1.5OP,HT1.5UP
2621,Switzerland,Lugano,Lausanne Sport,48.76,24.26,26.98,1-1,73.02,75.74,51.24,76.17,52.30,69.89,85.25,54.53,46.63,32.38,20.63,1-0,79.01,67.26,53.01,78.85,57.00,80.31,68.11,48.32,31.67,77.67,89.15,Lugano,Lausanne,2.20,3.5,3.00,1.36,1.29,1.67,2.88,2.25,3.50,1.29,1.57,1.40,1.62,2.20,1.22,4.00,1.73,2.08,2.75,1.40,5.50,1.14,1.33,3.25,2.63,1.44,2024-12-15 00:00:00,Switzerland,1 - 4,(0-2),1.0,4.0,5.0,0.0,2.0,2.0,0,0,1,0,1,1,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,1,1,0,1,0,0.0,-1.0,-1.0,-1.00,1.29,-1.0,1.22,-1.0,1.73,-1.00,-1.0,0.00,-1.0,0.00,1.62,-1.0,0.00,-1.0,-1.0,0.00,-1.0,-1.00,1.33,-1.0,-1.0,0.0
2622,Turkey,Hatayspor,Goztepe,17.23,27.10,55.65,1-1,44.33,72.88,82.75,80.37,54.61,67.71,83.71,55.63,30.98,43.25,25.53,0-0,74.23,56.51,68.78,74.75,50.87,62.89,84.65,26.11,55.90,92.11,71.04,Hatayspor,Goztepe,2.88,3.5,2.35,1.57,1.29,1.40,3.50,2.20,3.00,1.36,1.62,1.29,1.73,2.00,1.25,3.75,1.85,2.00,3.00,1.36,6.00,1.13,1.40,2.75,2.75,1.40,2024-12-15 00:00:00,Turkey,1 - 1,(0-1),1.0,1.0,2.0,0.0,1.0,1.0,0,1,0,1,0,1,1,0,0,1,0,1,0,1,1,0,0,0,1,0,1,1,1,0,0,1,-1.0,-1.0,0.0,-1.00,-1.00,1.4,1.25,-1.0,0.00,-1.00,-1.0,1.36,-1.0,1.13,1.73,-1.0,-1.00,0.0,-1.0,0.00,-1.0,-1.00,1.40,-1.0,-1.0,1.4
2623,Turkey,Antalyaspor,Kayserispor,51.51,26.04,22.43,1-1,77.55,73.94,48.47,86.08,63.68,58.42,76.57,64.76,28.67,46.54,24.68,0-0,75.21,53.35,71.22,69.31,59.55,86.34,72.47,59.16,36.97,67.88,85.91,Antalyaspor,Kayserispor,2.10,3.5,3.30,1.33,1.29,1.70,2.75,2.20,3.75,1.25,1.62,1.40,1.67,2.10,1.25,3.75,1.85,2.00,3.00,1.36,6.00,1.13,1.36,3.00,2.75,1.40,2024-12-15 00:00:00,Turkey,2 - 0,(1-0),2.0,0.0,2.0,1.0,0.0,1.0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,0,1,1,0,0,1,1,0,1,0,0,1,2.1,-1.0,-1.0,1.33,-1.00,-1.0,1.25,-1.0,0.00,-1.00,-1.0,1.36,-1.0,1.13,0.00,-1.0,-1.00,0.0,-1.0,1.25,-1.0,-1.00,1.36,-1.0,-1.0,1.4
2624,Turkey,Fenerbahce,Basaksehir,73.45,17.43,8.90,2-0,90.88,82.35,26.33,89.73,71.47,49.12,68.45,58.88,63.97,29.71,4.71,1-0,93.68,68.68,34.42,78.42,46.85,93.45,61.32,76.00,24.65,47.88,92.60,Fenerbahce,Basaksehir,1.40,5.5,6.00,1.13,1.14,2.75,1.91,2.50,6.50,1.10,1.50,1.80,1.83,1.83,1.18,4.50,1.57,2.35,2.50,1.50,4.33,1.20,1.30,3.40,2.38,1.53,2024-12-15 00:00:00,Turkey,3 - 1,(1-0),3.0,1.0,4.0,1.0,0.0,1.0,1,0,0,1,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,1,0,1,0,0,1,1.4,-1.0,-1.0,1.13,-1.00,-1.0,1.18,-1.0,1.57,-1.00,2.5,-1.00,-1.0,1.20,1.83,-1.0,1.91,-1.0,-1.0,1.10,-1.0,-1.00,1.30,-1.0,0.0,-1.0
2625,Scotland2,Hamilton,Partick Thistle,0.24,0.47,40.89,1-8,0.71,41.13,41.36,41.56,41.44,0.61,1.83,31.40,0.05,0.18,7.15,0-4,0.23,7.20,7.33,7.37,0.11,31.41,41.58,17.08,41.50,34.59,0.49,Hamilton,Partick Thistle,2.90,3.6,2.10,1.60,1.22,1.30,3.50,2.20,2.75,1.40,1.62,1.29,1.67,2.10,1.22,4.00,1.75,2.05,2.75,1.40,5.00,1.14,1.36,3.00,2.63,1.44,2024-09-28 00:00:00,Scotland2,1 - 0,(0-0),1.0,0.0,1.0,0.0,0.0,0.0,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,-1.0,-1.0,0.0,-1.00,-1.00,0.0,-1.00,4.0,-1.00,2.05,0.0,-1.00,0.0,-1.00,-1.00,2.1,-1.00,-1.0,0.0,-1.00,-1.0,1.29,-1.00,3.0,0.0,-1.0


# Checking For ROI of Profit Columns

In [7]:
# Select columns that end with 'P'
columns = [col for col in final_df_unique.columns if col.endswith('P')]

# Initialize lists to store results and games
results, games_list = [],  []

for col in columns:
    my_df = final_df_unique[final_df_unique[col] >= 0]
    numerator = np.sum(my_df[col]) - len(my_df)
    result = round(numerator / len(my_df) * 100, 2)
    
    # Append results and games to respective lists
    results.append(result)
    games_list.append(len(my_df))

# Convert results to a DataFrame with an additional column for Games
results_df = pd.DataFrame({
    'Column': columns,
    'ROI': results,
    'Games': games_list
})
results_df

,Column,ROI,Games
0,FT1P,-4.47,1471
1,FTXP,-6.80,228
2,FT2P,-15.05,927
3,FT1XP,-2.76,1161
4,FT12P,-5.51,754
5,FTX2P,-7.96,711
6,1.5OP,-5.38,2382
7,1.5UP,4.46,244
8,2.5OP,-5.53,1278
9,2.5UP,-5.69,1348


# ROI of Profit Columns According To Leagues

In [8]:
# Step 1: Filter leagues with at least 10 games
league_counts = final_df_unique['League'].value_counts()
leagues_with_10_games = league_counts[league_counts >= 10].index
filtered_df = final_df_unique[final_df_unique['League'].isin(leagues_with_10_games)]

# Group by 'League' and calculate results for each group
grouped_results = {}
for league, group in filtered_df.groupby('League'):
    group_results = {}
    for col in columns:
        my_df = group[group[col] >= 0]
        numerator = np.sum(my_df[col]) - len(my_df)
        group_results[col] = round(numerator / len(my_df) * 100, 2)
    # Add the number of games for this league
    group_results['Games'] = round(len(group),2)
    grouped_results[league] = group_results

# Convert grouped results to a DataFrame for better visualization
grouped_results_df = pd.DataFrame(grouped_results).T

# Define a function to apply conditional formatting
def highlight_positive(val):
    # Highlight background to red if the value is positive
    color = 'background-color: red' if isinstance(val, (int, float)) and val > 0 else ''
    return color

# Apply the function to the DataFrame
styled_df = (
    grouped_results_df.style
    .applymap(highlight_positive)
    .format("{:.2f}")  # Format only numeric columns, excluding 'Games'
)

# Save the styled DataFrame to Excel
styled_df.to_excel("ROI_leagues.xlsx", index=True)

# Display the styled DataFrame
styled_df

,FT1P,FTXP,FT2P,FT1XP,FT12P,FTX2P,1.5OP,1.5UP,2.5OP,2.5UP,3.5OP,3.5UP,4.5OP,4.5UP,BTTSP,OTTSP,HT1P,HTXP,HT2P,HT1XP,HT12P,HTX2P,HT0.5OP,HT0.5UP,HT1.5OP,HT1.5UP,Games
Austria,-26.90,-100.00,6.80,-9.71,13.41,-32.80,-1.48,300.00,16.68,-17.77,-2.92,14.12,-100.00,2.75,-18.65,-7.00,-51.20,0.25,64.17,1.23,nan,7.05,-3.70,21.50,9.94,17.32,53.00
Belgium,1.74,7.00,-45.25,8.07,-7.64,-20.61,1.00,-59.38,-10.66,-12.62,-33.00,3.72,-100.00,0.04,-27.09,-0.67,19.23,-8.34,17.05,-1.85,-70.00,-1.59,-1.10,-21.55,-74.00,-11.97,83.00
Brazil,-22.69,-100.00,-17.67,-4.39,2.31,-5.70,-4.65,175.00,-3.96,-12.17,nan,-4.46,nan,-1.31,0.31,-20.74,-17.08,4.38,nan,-6.77,nan,6.48,-1.29,41.29,-100.00,-9.69,105.00
Denmark,7.81,75.00,-57.17,10.53,-22.62,-11.25,-1.24,nan,-9.97,-2.00,0.44,-10.00,33.33,-1.70,7.15,-1.62,45.28,-5.00,-64.44,11.96,-24.25,-0.19,0.52,50.00,23.79,11.19,46.00
England,-10.61,-8.33,-32.94,-1.37,24.29,-6.36,-4.33,-100.00,-2.60,-4.60,-31.40,-8.07,-100.00,-2.09,8.91,-13.34,-21.41,6.39,28.33,-7.76,28.75,-14.53,-0.24,-14.29,-1.32,3.84,83.00
England2,-7.18,-5.71,-35.78,3.98,-2.12,1.80,-10.49,36.30,-20.02,8.65,-83.00,-0.15,-100.00,2.42,-16.52,0.69,19.05,-14.98,-77.27,-5.56,nan,-20.44,-7.19,9.84,6.77,0.85,133.00
England3,12.40,-2.00,-12.96,-8.78,-2.08,-11.56,-4.78,-10.88,9.11,5.09,-33.35,-4.05,-58.33,-5.60,1.10,4.91,-12.45,-24.20,4.13,-6.59,15.00,-10.71,-2.29,-20.12,0.36,-9.39,138.00
England4,-31.83,63.75,-2.44,-9.27,-10.66,-5.20,-14.08,0.47,-17.27,0.18,9.58,3.57,116.67,3.12,-25.49,-12.50,-18.76,-4.06,-42.50,-14.10,-26.80,-17.27,-11.12,-7.24,-17.74,-1.49,119.00
England5,-4.90,-100.00,-29.00,-0.11,-14.31,-14.03,-4.53,-59.38,-8.78,-26.19,-53.60,-12.86,-100.00,-3.38,-12.28,-25.34,3.45,-20.15,-7.67,-6.35,-41.37,-10.83,-4.24,-56.69,-18.00,-7.01,126.00
France,10.82,-34.00,-6.48,-7.82,1.71,-10.36,-0.48,-100.00,-1.14,-0.97,-16.86,-4.53,-38.14,3.14,-22.32,-11.29,-8.93,-37.68,-14.14,-15.82,53.00,-23.61,0.20,-100.00,32.92,0.48,76.00


# Creating Optimum Threshold for Each Prediction Column

In [9]:
# Assuming `df` is your DataFrame and it contains the columns for percentages and correctness
def calculate_threshold(percentages, predictions):
    # Ensure inputs are pandas Series
    percentages = pd.Series(percentages)
    predictions = pd.Series(predictions)
    
    thresholds = percentages.unique()
    best_threshold = 0
    best_j_stat = -np.inf  # Start with negative infinity for comparison
    
    for threshold in thresholds:
        # Predict 1s based on the threshold
        predicted_1s = (percentages >= threshold).astype(int)
        
        # Calculate true positives, true negatives, false positives, false negatives
        true_positives = ((predicted_1s == 1) & (predictions == 1)).sum()
        true_negatives = ((predicted_1s == 0) & (predictions == 0)).sum()
        false_positives = ((predicted_1s == 1) & (predictions == 0)).sum()
        false_negatives = ((predicted_1s == 0) & (predictions == 1)).sum()
        
        # Calculate Sensitivity (Recall) and Specificity
        sensitivity = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
        specificity = true_negatives / (true_negatives + false_positives) if (true_negatives + false_positives) > 0 else 0
        
        # Calculate Youden's J Statistic
        j_stat = sensitivity + specificity - 1
        
        # Update best threshold if J statistic improves
        if j_stat > best_j_stat:
            best_j_stat = j_stat
            best_threshold = threshold
    
    return best_threshold, round(best_j_stat, 2)

# Select only numeric columns
numeric_columns = final_df_unique.select_dtypes(include=[np.number])

# Remove rows where any numeric value is greater than 100
final_df_unique = final_df_unique[(numeric_columns <= 100).all(axis=1)]

#Selecting dataframes with model predictions
ft1df = final_df_unique[final_df_unique['df1_FT1'] >= final_df_unique[['df1_FTX', 'df1_FT2']].max(axis=1)]
ftxdf = final_df_unique[final_df_unique['df1_FTX'] >= final_df_unique[['df1_FT1', 'df1_FT2']].max(axis=1)]
ft2df = final_df_unique[final_df_unique['df1_FT2'] >= final_df_unique[['df1_FTX', 'df1_FT1']].max(axis=1)]
dc1xdf = final_df_unique[final_df_unique['df1_DC1X'] >= final_df_unique[['df1_DC12', 'df1_DCX2']].max(axis=1)]
dc12df = final_df_unique[final_df_unique['df1_DC12'] >= final_df_unique[['df1_DC1X', 'df1_DCX2']].max(axis=1)]
dcx2df = final_df_unique[final_df_unique['df1_DCX2'] >= final_df_unique[['df1_DC1X', 'df1_DC12']].max(axis=1)]
over15df, under15df = final_df_unique[final_df_unique['df1_1.5O'] >= 50], final_df_unique[final_df_unique['df1_1.5O'] < 50]
over25df, under25df = final_df_unique[final_df_unique['df1_2.5O'] >= 50], final_df_unique[final_df_unique['df1_2.5O'] < 50]
over35df, under35df = final_df_unique[final_df_unique['df1_3.5U'] < 50], final_df_unique[final_df_unique['df1_3.5U'] >= 50]
over45df, under45df = final_df_unique[final_df_unique['df1_4.5U'] < 50], final_df_unique[final_df_unique['df1_4.5U'] >= 50]
bttsdf, ottsdf = final_df_unique[final_df_unique['df1_BTTS'] >= 50], final_df_unique[final_df_unique['df1_BTTS'] < 50]
ht1df = final_df_unique[final_df_unique['df1_HT1'] >= final_df_unique[['df1_HTX', 'df1_HT2']].max(axis=1)]
htxdf = final_df_unique[final_df_unique['df1_HTX'] >= final_df_unique[['df1_HT1', 'df1_HT2']].max(axis=1)]
ht2df = final_df_unique[final_df_unique['df1_HT2'] >= final_df_unique[['df1_HT1', 'df1_HTX']].max(axis=1)]
ht1xdf = final_df_unique[final_df_unique['df1_HTDC1X'] >= final_df_unique[['df1_HTDC12', 'df1_HTDCX2']].max(axis=1)]
ht12df = final_df_unique[final_df_unique['df1_HTDC12'] >= final_df_unique[['df1_HTDC1X', 'df1_HTDCX2']].max(axis=1)]
htx2df = final_df_unique[final_df_unique['df1_HTDCX2'] >= final_df_unique[['df1_HTDC1X', 'df1_HTDC12']].max(axis=1)]
htover05df, htunder05df = final_df_unique[final_df_unique['df1_HT0.5O'] >= 50], final_df_unique[final_df_unique['df1_HT0.5O'] < 50]
htover15df, htunder15df = final_df_unique[final_df_unique['df1_HT1.5U'] < 50], final_df_unique[final_df_unique['df1_HT1.5U'] >= 50]

ft1t, ft1a = calculate_threshold(ft1df['df1_FT1'], ft1df['FT1'])
ftxt, ftxa = calculate_threshold(ftxdf['df1_FTX'], ftxdf['FTX'])
ft2t, ft2a = calculate_threshold(ft2df['df1_FT2'], ft2df['FT2'])
ft1xt, ft1xa = calculate_threshold(dc1xdf['df1_DC1X'], dc1xdf['FT1X'])
ft12t, ft12a = calculate_threshold(dc12df['df1_DC12'], dc12df['FT12'])
ftx2t, ftx2a = calculate_threshold(dcx2df['df1_DCX2'], dcx2df['FTX2'])
over15t, over15a = calculate_threshold(over15df['df1_1.5O'], over15df['1.5O'])
under15t, under15a = calculate_threshold(under15df['df1_1.5O'], under15df['1.5U'])
over25t, over25a = calculate_threshold(over25df['df1_2.5O'], over25df['2.5O'])
under25t, under25a = calculate_threshold(under25df['df1_2.5O'], under25df['2.5U'])
over35t, over35a = calculate_threshold(over35df['df1_3.5U'], over35df['3.5O'])
under35t, under35a = calculate_threshold(under35df['df1_3.5U'], under35df['3.5U'])
over45t, over45a = calculate_threshold(over45df['df1_4.5U'], over45df['4.5O'])
under45t, under45a = calculate_threshold(under45df['df1_4.5U'], under45df['4.5U'])
bttst, bttsa = calculate_threshold(bttsdf['df1_BTTS'], bttsdf['BTTS'])
ottst, ottsa = calculate_threshold(ottsdf['df1_BTTS'], ottsdf['OTTS'])
ht1t, ht1a = calculate_threshold(ht1df['df1_HT1'], ht1df['HT1'])
htxt, htxa = calculate_threshold(htxdf['df1_HTX'], htxdf['HTX'])
ht2t, ht2a = calculate_threshold(ht2df['df1_HT2'], ht2df['HT2'])
ht1xt, ht1xa = calculate_threshold(ht1xdf['df1_HTDC1X'], ht1xdf['HT1X'])
ht12t, ht12a = calculate_threshold(ht12df['df1_HTDC12'], ht12df['HT12'])
htx2t, htx2a = calculate_threshold(htx2df['df1_HTDCX2'], htx2df['HTX2'])
htover05t, htover05a = calculate_threshold(htover05df['df1_HT0.5O'], htover05df['HT0.5O'])
htunder05t, htunder05a = calculate_threshold(htunder05df['df1_HT0.5O'], htunder05df['HT0.5U'])
htover15t, htover15a = calculate_threshold(htover15df['df1_HT1.5U'], htover15df['HT1.5O'])
htunder15t, htunder15a = calculate_threshold(htunder15df['df1_HT1.5U'], htunder15df['HT1.5U'])

new_ft1df, new_ftxdf, new_ft2df = ft1df[ft1df['df1_FT1'] >= ft1t],ftxdf[ftxdf['df1_FTX'] >= ftxt],ft2df[ft2df['df1_FT2'] >= ft2t]
new_ft1xdf, new_ft12df, new_ftx2df = dc1xdf[dc1xdf['df1_DC1X'] >= ft1xt],dc12df[dc12df['df1_DC12'] >= ft12t],dcx2df[dcx2df['df1_DCX2'] >= ftx2t] 
new_over15, new_under15 = over15df[over15df['df1_1.5O'] >= over15t], under15df[under15df['df1_1.5O'] <= under15t]
new_over25, new_under25 = over25df[over25df['df1_2.5O'] >= over25t], under25df[under25df['df1_2.5O'] <= under25t]
new_over35, new_under35 = over35df[over35df['df1_3.5U'] <= over35t], under35df[under35df['df1_3.5U'] >= under35t]
new_over45, new_under45 = over45df[over45df['df1_4.5U'] <= over45t], under45df[under45df['df1_4.5U'] >= under45t]
new_btts, new_otts = bttsdf[bttsdf['df1_BTTS'] >= bttst], ottsdf[ottsdf['df1_BTTS'] <= ottst]
new_ht1df, new_htxdf, new_ht2df = ht1df[ht1df['df1_HT1'] >= ht1t],htxdf[htxdf['df1_HTX'] >= htxt],ht2df[ht2df['df1_HT2'] >= ht2t]
new_ht1xdf, new_ht12df, new_htx2df = ht1xdf[ht1xdf['df1_HTDC1X'] >= ht1xt],ht12df[ht12df['df1_HTDC12'] >= ht12t],htx2df[htx2df['df1_HTDCX2'] >= htx2t]
new_htover05, new_htunder05 = htover05df[htover05df['df1_HT0.5O'] >= htover05t], htunder05df[htunder05df['df1_HT0.5O'] <= htunder05t]
new_htover15, new_htunder15 = htover15df[htover15df['df1_HT1.5U'] <= htover15t], htunder15df[htunder15df['df1_HT1.5U'] >= htunder15t]

# Store the results in a list
results = [
('FT1', ft1t, ft1a, len(new_ft1df), round(len(new_ft1df)/len(ft1df)*100,2), np.sum(new_ft1df['FT1P']) - len(new_ft1df)),
('FTX', ftxt, ftxa, len(new_ftxdf), round(len(new_ftxdf)/len(ftxdf)*100,2), np.sum(new_ftxdf['FTXP']) - len(new_ftxdf)),
('FT2', ft2t, ft2a, len(new_ft2df), round(len(new_ft2df)/len(ft2df)*100,2), np.sum(new_ft2df['FT2P']) - len(new_ft2df)),
('FT1X', ft1xt, ft1xa, len(new_ft1xdf), round(len(new_ft1xdf)/len(dc1xdf)*100,2), np.sum(new_ft1xdf['FT1XP']) - len(new_ft1xdf)),
('FT12', ft12t, ft12a, len(new_ft12df), round(len(new_ft12df)/len(dc12df)*100,2), np.sum(new_ft12df['FT12P']) - len(new_ft12df)),
('FTX2', ftx2t, ftx2a, len(new_ftx2df), round(len(new_ftx2df)/len(dcx2df)*100,2), np.sum(new_ftx2df['FTX2P']) - len(new_ftx2df)),
('1.5O', over15t, over15a, len(new_over15), round(len(new_over15)/len(over15df)*100,2), np.sum(new_over15['1.5OP'])-len(new_over15)),
('1.5U', under15t, under15a, len(new_under15), round(len(new_under15)/len(under15df)*100,2), np.sum(new_under15['1.5UP'])-len(new_under15)),
('2.5O', over25t, over25a, len(new_over25), round(len(new_over25)/len(over25df)*100,2), np.sum(new_over25['2.5OP'])-len(new_over25)),
('2.5U', under25t, under25a, len(new_under25), round(len(new_under25)/len(under25df)*100,2), np.sum(new_under25['2.5UP'])-len(new_under25)),
('3.5O', over35t, over35a, len(new_over35), round(len(new_over35)/len(over35df)*100,2), np.sum(new_over35['3.5OP'])-len(new_over35)),
('3.5U', under35t, under35a, len(new_under35), round(len(new_under35)/len(under35df)*100,2), np.sum(new_under35['3.5UP'])-len(new_under35)),
('4.5O', over45t, over45a, len(new_over45), round(len(new_over45)/len(over45df)*100,2), np.sum(new_over45['4.5OP'])-len(new_over45)),
('4.5U', under45t, under45a, len(new_under45), round(len(new_under45)/len(under45df)*100,2), np.sum(new_under45['4.5UP'])-len(new_under45)),
('BTTS', bttst, bttsa, len(new_btts), round(len(new_btts)/len(bttsdf)*100,2), np.sum(new_btts['BTTSP'])-len(new_btts)),
('OTTS', ottst, ottsa, len(new_otts), round(len(new_otts)/len(ottsdf)*100,2), np.sum(new_otts['OTTSP'])-len(new_otts)),
('HT1', ht1t, ht1a, len(new_ht1df), round(len(new_ht1df)/len(ht1df)*100,2), np.sum(new_ht1df['HT1P']) - len(new_ht1df)),
('HTX', htxt, htxa, len(new_htxdf), round(len(new_htxdf)/len(htxdf)*100,2), np.sum(new_htxdf['HTXP']) - len(new_htxdf)),
('HT2', ht2t, ht2a, len(new_ht2df), round(len(new_ht2df)/len(ht2df)*100,2), np.sum(new_ht2df['HT2P']) - len(new_ht2df)),
('HT1X', ht1xt, ht1xa, len(new_ht1xdf), round(len(new_ht1xdf)/len(ht1xdf)*100,2), np.sum(new_ht1xdf['HT1XP']) - len(new_ht1xdf)),
('HT12', ht12t, ht12a, len(new_ht12df), round(len(new_ht12df)/len(ht12df)*100,2), np.sum(new_ht12df['HT12P']) - len(new_ht12df)),
('HTX2', htx2t, htx2a, len(new_htx2df), round(len(new_htx2df)/len(htx2df)*100,2), np.sum(new_htx2df['HTX2P']) - len(new_htx2df)),
('HT0.5O', htover05t, htover05a, len(new_htover05), round(len(new_htover05)/len(htover05df)*100,2), np.sum(new_htover05['HT0.5OP'])-len(new_htover05)),
('HT0.5U', htunder05t, htunder05a, len(new_htunder05), round(len(new_htunder05)/len(htunder05df)*100,2), np.sum(new_htunder05['HT0.5UP'])-len(new_htunder05)),
('HT1.5O', htover15t, htover15a, len(new_htover15), round(len(new_htover15)/len(htover15df)*100,2), np.sum(new_htover15['HT1.5OP'])-len(new_htover15)),
('HT1.5U', htunder15t, htunder15a, len(new_htunder15), round(len(new_htunder15)/len(htunder15df)*100,2), np.sum(new_htunder15['HT1.5UP'])-len(new_htunder15))
]

# Create a DataFrame from the results
results_df = pd.DataFrame(results, columns=['Prediction', 'Threshold', 'J-Stat', 'Games', 'Games%', 'Profit'])
results_df['ROI'] = round(results_df['Profit'] / results_df['Games'] * 100, 2)
print('Number of matches: ', len(final_df_unique))
results_df

Number of matches:  2580


,Prediction,Threshold,J-Stat,Games,Games%,Profit,ROI
0,FT1,59.31,0.19,618,42.62,20.65,3.34
1,FTX,45.38,0.10,84,39.81,8.10,9.64
2,FT2,60.99,0.12,240,26.09,-22.77,-9.49
3,FT1X,87.60,0.17,443,39.20,-0.66,-0.15
4,FT12,77.40,0.03,343,45.55,-15.03,-4.38
5,FTX2,82.40,0.13,361,51.79,-19.63,-5.44
6,1.5O,79.73,0.08,965,41.24,-40.70,-4.22
7,1.5U,38.43,0.09,89,37.08,-6.31,-7.09
8,2.5O,70.66,0.05,394,31.29,-12.62,-3.20
9,2.5U,47.50,0.01,1203,91.07,-82.16,-6.83


# Testing Best / Most Profitable Model Predictions

In [10]:
# Select columns starting with 'df1_'
predictions = ['df1_FT1', 'df1_FTX', 'df1_FT2', 'df1_DC1X', 'df1_DC12', 'df1_DCX2', 
               'df1_1.5O', 'df1_2.5O', 'df1_3.5U', 'df1_4.5U', 'df1_BTTS', 
               'df1_HT1', 'df1_HTX', 'df1_HT2', 'df1_HTDC1X', 'df1_HTDC12', 'df1_HTDCX2', 
               'df1_HT0.5O', 'df1_HT1.5U']

# Select columns starting with 'df2_'
results = ['FT1', 'FTX', 'FT2', 'FT1X', 'FT12', 'FTX2', 
           '1.5O', '2.5O', '3.5U', '4.5U', 'BTTS',
            'HT1', 'HTX', 'HT2', 'HT1X', 'HT12', 'HTX2', 
            'HT0.5O', 'df2_HT1.5U']

# Select columns ending with 'P'
profits = ['FT1P', 'FTXP', 'FT2P', 'FT1XP', 'FT12P', 'FTX2P', 
           '1.5OP', '2.5OP', '3.5UP', '4.5UP', 'BTTSP',
            'HT1P', 'HTXP', 'HT2P', 'HT1XP', 'HT12P', 'HTX2P', 
            'HT0.5OP', 'HT1.5UP']

bet, percentage, profit = [], [], []

for i in range(len(final_df_unique)):
    my_list = []
    for j in predictions:
        my_list.append(final_df_unique[j].iloc[i])
    percentage.append(max(my_list))
    max_index = my_list.index(max(my_list))
    bet.append(results[max_index])
    profit_column = profits[max_index]
    profit.append(final_df_unique[profit_column].iloc[i])

# Create a DataFrame
model_recs = pd.DataFrame({
    'League': final_df_unique['df1_League'],
    'Home': final_df_unique['Home'],
    'Away': final_df_unique['Away'],
    'BET': bet,
    'Percentage': percentage,
    'Profit': profit
})


print('Matches found: ', len(final_df_unique))
print(f"Correct Predictions: {len(model_recs[model_recs['Profit'] > 0])/len(model_recs)*100}")
print(f"Profit: {round(sum(model_recs['Profit']) - len(model_recs),2)} ROI: {round((sum(model_recs['Profit']) - len(model_recs)) / len(model_recs) * 100, 2)}%")
model_recs.tail()

Matches found:  2580
Correct Predictions: 83.4108527131783
Profit: -84.53 ROI: -3.28%


,League,Home,Away,BET,Percentage,Profit
2621,Switzerland,Lugano,Lausanne Sport,4.5U,85.25,0.00
2622,Turkey,Hatayspor,Goztepe,4.5U,83.71,1.13
2623,Turkey,Antalyaspor,Kayserispor,1.5O,86.08,1.25
2624,Turkey,Fenerbahce,Basaksehir,HT1X,93.68,1.10
2625,Scotland2,Hamilton,Partick Thistle,1.5O,41.56,-1.00


In [11]:
final_df_unique['df1_OTTS'] = 100 - final_df_unique['df1_BTTS']
final_df_unique['df1_1.5U'] = 100 - final_df_unique['df1_1.5O']
final_df_unique['df1_2.5U'] = 100 - final_df_unique['df1_2.5O']
final_df_unique['df1_3.5O'] = 100 - final_df_unique['df1_3.5U']
final_df_unique['df1_4.5O'] = 100 - final_df_unique['df1_4.5U']
final_df_unique['df1_HT0.5U'] = 100 - final_df_unique['df1_HT0.5O']
final_df_unique['df1_HT1.5O'] = 100 - final_df_unique['df1_HT1.5U'] 

# Select columns starting with 'df1_'
predictions = ['df1_FT1', 'df1_FTX', 'df1_FT2', 'df1_DC1X', 'df1_DC12', 'df1_DCX2', 
               'df1_1.5O', 'df1_1.5U', 'df1_2.5O','df1_2.5U','df1_3.5O', 'df1_3.5U', 
               'df1_4.5O', 'df1_4.5U', 'df1_BTTS', 'df1_OTTS',
               'df1_HT1', 'df1_HTX', 'df1_HT2', 'df1_HTDC1X', 'df1_HTDC12', 'df1_HTDCX2', 
               'df1_HT0.5O', 'df1_HT0.5U', 'df1_HT1.5O', 'df1_HT1.5U']

# Select columns starting with 'df2_'
odds = ['df2_FT1', 'df2_FTX', 'df2_FT2', 'df2_DC1X', 'df2_DC12', 'df2_DCX2', 
           'df2_1.5O', 'df2_1.5U', 'df2_2.5O','df2_2.5U', 'df2_3.5O','df2_3.5U', 
           'df2_4.5O', 'df2_4.5U', 'df2_BTTS', 'df2_OTTS',
            'df2_HT1', 'df2_HTX', 'df2_HT2', 'df2_HT1X', 'df2_HT12', 'df2_HTX2', 
            'df2_HT0.5O', 'df2_HT0.5U', 'df2_HT1.5O', 'df2_HT1.5U']

# Select columns ending with 'P'
profit = ['FT1P', 'FTXP', 'FT2P', 'FT1XP', 'FT12P', 'FTX2P', 
           '1.5OP', '1.5UP', '2.5OP', '2.5UP', '3.5OP', '3.5UP', '4.5OP', '4.5UP', 
           'BTTSP', 'OTTSP', 'HT1P', 'HTXP', 'HT2P', 'HT1XP', 'HT12P', 'HTX2P', 
            'HT0.5OP', 'HT0.5UP', 'HT1.5OP','HT1.5UP']

bets, percentages, profits, difference = [], [], [], [] 
for i in range(len(final_df_unique)):
    my_list = []
    valid_indices = []  # To keep track of indices where profit is not negative
    for j in range(len(predictions)):
        pred_column = predictions[j]
        odds_column = odds[j]
        profit_column = profit[j]  # Corresponding profit column
        
        # Calculate the value
        my_value = (100 / final_df_unique[pred_column].iloc[i]) - final_df_unique[odds_column].iloc[i]
        
        # Only add to the list if the corresponding profit is non-negative
        if final_df_unique[profit_column].iloc[i] >= 0:
            my_list.append(my_value)
            valid_indices.append(j)
        else:
            my_list.append(float('-inf'))  # Set to negative infinity to ignore in max()
    
    if valid_indices:  # Ensure there is at least one valid index
        max_index = my_list.index(max(my_list))
        rec_bet = profit[max_index]
        percent_bet = predictions[max_index]
        bets.append(rec_bet)
        percentages.append(final_df_unique[percent_bet].iloc[i])
        rec_profit = profit[max_index]
        profits.append(final_df_unique[rec_profit].iloc[i])
        difference.append(round(max(my_list), 2))
    else:
        # Handle case where no valid profits are found for this match
        bets.append(None)
        percentages.append(None)
        profits.append(None)
        difference.append(None)

# Create a DataFrame
model_recs = pd.DataFrame({
    'League': final_df_unique['df1_League'],
    'Home': final_df_unique['Home'],
    'Away': final_df_unique['Away'],
    'BET': bets,
    'Percentage': percentages,
    'Profit': profits,
    'Difference': difference
}).dropna()  # Drop rows with None values

print('Matches found: ', len(final_df_unique))
print(f"Correct Predictions: {len(model_recs[model_recs['Profit'] > 0])/len(model_recs)*100}")
print(f"Profit: {round(sum(model_recs['Profit']) - len(model_recs), 2)} ROI: {round((sum(model_recs['Profit']) - len(model_recs)) / len(model_recs) * 100, 2)}%")
model_recs.tail()

Matches found:  2580
Correct Predictions: 69.92248062015504
Profit: -63.09 ROI: -2.45%


,League,Home,Away,BET,Percentage,Profit,Difference
2621,Switzerland,Lugano,Lausanne Sport,HT1.5UP,57.00,0.00,0.31
2622,Turkey,Hatayspor,Goztepe,HT1.5UP,50.87,1.40,0.57
2623,Turkey,Antalyaspor,Kayserispor,3.5UP,58.42,1.36,0.35
2624,Turkey,Fenerbahce,Basaksehir,4.5UP,68.45,1.20,0.26
2625,Scotland2,Hamilton,Partick Thistle,HTX2P,7.33,1.29,12.35


# Combining Different Bets

In [12]:
# Define the list of base columns and multiplier columns
base_columns = ['df1_FT1', 'df1_FTX', 'df1_FT2']  
multiplier_columns = ['df1_1.5O', 'df1_2.5O', 'df1_2.5U', 'df1_3.5U', 'df1_4.5U']

# Nested loop: Iterate over each base column and multiplier column
for base_column in base_columns:
    for multiplier in multiplier_columns:
        # Create a new column name dynamically
        new_column = f"{base_column}/{multiplier.split('_')[1]}"
        
        # Perform the calculation and assign it to the new column
        final_df_unique[new_column] = round(final_df_unique[base_column] * final_df_unique[multiplier] / 100, 2)

final_df_unique.tail()

,df1_League,Home,Away,df1_FT1,df1_FTX,df1_FT2,df1_FTR,df1_DC1X,df1_DC12,df1_DCX2,df1_1.5O,df1_2.5O,df1_3.5U,df1_4.5U,df1_BTTS,df1_HT1,df1_HTX,df1_HT2,df1_HTR,df1_HTDC1X,df1_HTDC12,df1_HTDCX2,df1_HT0.5O,df1_HT1.5U,df1_H0.5O,df1_A0.5O,df1_H1.5O,df1_A1.5O,df1_H2.5U,df1_A2.5U,df2_Home,df2_Away,df2_FT1,df2_FTX,df2_FT2,df2_DC1X,df2_DC12,df2_DCX2,df2_HT1,df2_HTX,df2_HT2,df2_HT1X,df2_HT12,df2_HTX2,df2_BTTS,df2_OTTS,df2_1.5O,df2_1.5U,df2_2.5O,df2_2.5U,df2_3.5O,df2_3.5U,df2_4.5O,df2_4.5U,df2_HT0.5O,df2_HT0.5U,df2_HT1.5O,df2_HT1.5U,Date,League,FT,HT,FTHG,FTAG,FTTG,HTHG,HTAG,HTTG,FT1,FTX,FT2,FT1X,FT12,FTX2,1.5O,1.5U,2.5O,2.5U,3.5O,3.5U,4.5O,4.5U,BTTS,OTTS,HT1,HTX,HT2,HT1X,HT12,HTX2,HT0.5O,HT0.5U,HT1.5O,HT1.5U,FT1P,FTXP,FT2P,FT1XP,FT12P,FTX2P,1.5OP,1.5UP,2.5OP,2.5UP,3.5OP,3.5UP,4.5OP,4.5UP,BTTSP,OTTSP,HT1P,HTXP,HT2P,HT1XP,HT12P,HTX2P,HT0.5OP,HT0.5UP,HT1.5OP,HT1.5UP,df1_OTTS,df1_1.5U,df1_2.5U,df1_3.5O,df1_4.5O,df1_HT0.5U,df1_HT1.5O,df1_FT1/1.5O,df1_FT1/2.5O,df1_FT1/2.5U,df1_FT1/3.5U,df1_FT1/4.5U,df1_FTX/1.5O,df1_FTX/2.5O,df1_FTX/2.5U,df1_FTX/3.5U,df1_FTX/4.5U,df1_FT2/1.5O,df1_FT2/2.5O,df1_FT2/2.5U,df1_FT2/3.5U,df1_FT2/4.5U
2621,Switzerland,Lugano,Lausanne Sport,48.76,24.26,26.98,1-1,73.02,75.74,51.24,76.17,52.30,69.89,85.25,54.53,46.63,32.38,20.63,1-0,79.01,67.26,53.01,78.85,57.00,80.31,68.11,48.32,31.67,77.67,89.15,Lugano,Lausanne,2.20,3.5,3.00,1.36,1.29,1.67,2.88,2.25,3.50,1.29,1.57,1.40,1.62,2.20,1.22,4.00,1.73,2.08,2.75,1.40,5.50,1.14,1.33,3.25,2.63,1.44,2024-12-15 00:00:00,Switzerland,1 - 4,(0-2),1.0,4.0,5.0,0.0,2.0,2.0,0,0,1,0,1,1,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,1,1,0,1,0,0.0,-1.0,-1.0,-1.00,1.29,-1.0,1.22,-1.0,1.73,-1.00,-1.0,0.00,-1.0,0.00,1.62,-1.0,0.00,-1.0,-1.0,0.00,-1.0,-1.00,1.33,-1.0,-1.0,0.0,45.47,23.83,47.70,30.11,14.75,21.15,43.00,37.14,25.50,23.26,34.08,41.57,18.48,12.69,11.57,16.96,20.68,20.55,14.11,12.87,18.86,23.00
2622,Turkey,Hatayspor,Goztepe,17.23,27.10,55.65,1-1,44.33,72.88,82.75,80.37,54.61,67.71,83.71,55.63,30.98,43.25,25.53,0-0,74.23,56.51,68.78,74.75,50.87,62.89,84.65,26.11,55.90,92.11,71.04,Hatayspor,Goztepe,2.88,3.5,2.35,1.57,1.29,1.40,3.50,2.20,3.00,1.36,1.62,1.29,1.73,2.00,1.25,3.75,1.85,2.00,3.00,1.36,6.00,1.13,1.40,2.75,2.75,1.40,2024-12-15 00:00:00,Turkey,1 - 1,(0-1),1.0,1.0,2.0,0.0,1.0,1.0,0,1,0,1,0,1,1,0,0,1,0,1,0,1,1,0,0,0,1,0,1,1,1,0,0,1,-1.0,-1.0,0.0,-1.00,-1.00,1.4,1.25,-1.0,0.00,-1.00,-1.0,1.36,-1.0,1.13,1.73,-1.0,-1.00,0.0,-1.0,0.00,-1.0,-1.00,1.40,-1.0,-1.0,1.4,44.37,19.63,45.39,32.29,16.29,25.25,49.13,13.85,9.41,7.82,11.67,14.42,21.78,14.80,12.30,18.35,22.69,44.73,30.39,25.26,37.68,46.58
2623,Turkey,Antalyaspor,Kayserispor,51.51,26.04,22.43,1-1,77.55,73.94,48.47,86.08,63.68,58.42,76.57,64.76,28.67,46.54,24.68,0-0,75.21,53.35,71.22,69.31,59.55,86.34,72.47,59.16,36.97,67.88,85.91,Antalyaspor,Kayserispor,2.10,3.5,3.30,1.33,1.29,1.70,2.75,2.20,3.75,1.25,1.62,1.40,1.67,2.10,1.25,3.75,1.85,2.00,3.00,1.36,6.00,1.13,1.36,3.00,2.75,1.40,2024-12-15 00:00:00,Turkey,2 - 0,(1-0),2.0,0.0,2.0,1.0,0.0,1.0,1,0,0,1,1,0,1,0,0,1,0,1,0,1,0,1,1,0,0,1,1,0,1,0,0,1,2.1,-1.0,-1.0,1.33,-1.00,-1.0,1.25,-1.0,0.00,-1.00,-1.0,1.36,-1.0,1.13,0.00,-1.0,-1.00,0.0,-1.0,1.25,-1.0,-1.00,1.36,-1.0,-1.0,1.4,35.24,13.92,36.32,41.58,23.43,30.69,40.45,44.34,32.80,18.71,30.09,39.44,22.42,16.58,9.46,15.21,19.94,19.31,14.28,8.15,13.10,17.17
2624,Turkey,Fenerbahce,Basaksehir,73.45,17.43,8.90,2-0,90.88,82.35,26.33,89.73,71.47,49.12,68.45,58.88,63.97,29.71,4.71,1-0,93.68,68.68,34.42,78.42,46.85,93.45,61.32,76.00,24.65,47.88,92.60,Fenerbahce,Basaksehir,1.40,5.5,6.00,1.13,1.14,2.75,1.91,2.50,6.50,1.10,1.50,1.80,1.83,1.83,1.18,4.50,1.57,2.35,2.50,1.50,4.33,1.20,1.30,3.40,2.38,1.53,2024-12-15 00:00:00,Turkey,3 - 1,(1-0),3.0,1.0,4.0,1.0,0.0,1.0,1,0,0,1,1,0,1,0,1,0,1,0,0,1,1,0,1,0,0,1,1,0,1,0,0,1,1.4,-1.0,-1.0,1.13,-1.00,-1.0,1.18,-1.0,1.57,-1.00,2.5,-1.00,-1.0,1.20,1.83,-1.0,1.91,-1.0,-1.0,1.10,-1.0,-1.00,1.30,-1.0,0.0,-1.0,41.12,10.27,28.53,50.88,31.55,21.58,53.15,65.91,52.49,20.96,36.08,50.28,15.64,12.46,4.97,8.56,11.93,7.99,6.36,2.54,4.37,6.09
26